In [55]:
import numpy as np
import pandas as pd
import datetime
from sklearn.cross_validation import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

from sklearn import svm,tree
from sklearn.externals.six import StringIO  
import pydot

In [77]:
df = pd.read_csv('../data/aden_golfe_matrix_area.csv',sep=';')


In [78]:
def bool_attack(count):
    if count>0:
        return 1
    else:
        return 0

    
df['attack'] = df.apply(lambda row: bool_attack(row['count']), axis=1)
#area = df.apply(lambda row: str(row['lat_area'])+str(row['lon_area']), axis=1)
#print (area.values).astype(str)
#df['area']=(area.values).astype(str)
print df[df['count']!=0].head()
#print df.head()
#print df.dtypes
print np.sort(df['lat_area'].unique())

     Month  Years  count  lat_area  lon_area  attack
132      2   2008      2         4        20       1
207      3   2008      1         4        18       1
212      3   2008      1         6        19       1
219      3   2008      1         3        21       1
281      4   2008      1         4        16       1
[ 1  2  3  4  5  6  7  8  9 10]


# Binary

In [80]:
latUnique=np.sort(df['lat_area'].unique())
for i in range(0,len(latUnique)):
    nameColLat='lat_area_'+str(latUnique[i])
    #print nameColLat
    df[nameColLat]=0

print "-----------------"
    
lonUnique=np.sort(df['lon_area'].unique())
for i in range(0,len(lonUnique)):
    nameColLon='lon_area_'+str(lonUnique[i])
    #print nameColLon
    df[nameColLon]=0
    
    
print "-----------------"

for index, row in df.iterrows():
    if(row['count']>0):
        df.set_value(index, "lat_area_"+str(row['lat_area']),1)
        df.set_value(index, "lon_area_"+str(row['lon_area']),1)
        
    
df.head(10)

-----------------
-----------------


,Month,Years,count,lat_area,lon_area,attack,lat_area_1,lat_area_2,lat_area_3,lat_area_4,...,lon_area_19,lon_area_20,lon_area_21,lon_area_22,lon_area_23,lon_area_24,lon_area_25,lon_area_26,lon_area_27,lon_area_28
0,1,2008,0,8,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2008,0,9,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2008,0,6,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2008,0,7,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2008,0,8,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,2008,0,5,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2008,0,6,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,2008,0,7,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,2008,0,4,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,2008,0,5,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
features=['Month','Years',
          'lat_area_1','lat_area_2','lat_area_3',
          'lat_area_4','lat_area_5','lat_area_6','lat_area_7',
          'lat_area_8','lat_area_9','lat_area_10','lon_area_2','lon_area_3','lon_area_4','lon_area_5',
          'lon_area_6','lon_area_7','lon_area_8','lon_area_9','lon_area_10','lon_area_11','lon_area_12',
          'lon_area_13','lon_area_14','lon_area_15','lon_area_16','lon_area_17','lon_area_18','lon_area_19',
          'lon_area_20','lon_area_21','lon_area_22','lon_area_23','lon_area_24','lon_area_25','lon_area_26',
          'lon_area_27','lon_area_28']

target='attack'

In [93]:
dfTrain=df[df['Years']<2010]
dfTest=df[(df['Years']==2010)]
print len(dfTrain[dfTrain['count']!=0])
print len(dfTest[dfTest['count']!=0])

print len(dfTrain)
print len(dfTest)

155
56
1968
984


# Forest

In [94]:
forest = RandomForestClassifier()
forest=forest.fit(dfTrain[features], dfTrain[target])

predictions=forest.predict(dfTest[features])
print predictions.sum()

54


In [95]:
pd.crosstab(dfTest[target],predictions, colnames=['preds'])

preds,0,1
attack,,
0,928,0
1,2,54


# Compare area

In [96]:
dfTest['prediction']=predictions
dfTest[(dfTest[target]!=0) | (dfTest['prediction']!=0)].sort(['Years','Month'])

/Users/romainphilippe/app/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/romainphilippe/app/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,Month,Years,count,lat_area,lon_area,attack,lat_area_1,lat_area_2,lat_area_3,lat_area_4,...,lon_area_20,lon_area_21,lon_area_22,lon_area_23,lon_area_24,lon_area_25,lon_area_26,lon_area_27,lon_area_28,prediction
1994,1,2010,1,3,13,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1995,1,2010,1,4,13,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2000,1,2010,1,4,15,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2061,2,2010,1,3,6,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2075,2,2010,1,4,12,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2082,2,2010,1,4,15,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2137,3,2010,1,5,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2141,3,2010,1,5,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2147,3,2010,1,3,8,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2148,3,2010,1,4,8,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
